# Homeworks for SM-I course

Important: 

1). Please download the latest grcodes, images, and related chapters before working on the homework. 

2). Both pdf files and the source codes must be submitted, or the work will not be marked.  

# Homework 5: Stressed bars by mechanical and thermal loads

**Question 1:**

Consider a uniform bar with cross-section area of $A$, and length $l$ made of linear elastic material. It is subjected to a distributed body-force $b_f$, as shown in Fig.\ref{image:bar1Dbf}. It is fixed at the left-end, meaning that at $x=0, u=0$. At $x=l$, it may also be subjected to a concentrated force $f$. The problem can be treated as one-dimensional (1D), meaning that our concern is mainly the unknown displacement, stress, and strain varying with the coordinate $x$. 

<img src="/images/bar1Dbf.png" alt=" " width="450"/><br>


1) Consider only the body force $b_f=q$ (N/m) where $q$ is a constant, derive by hand the formulas for computing the displacement in the $x$-direction, and normal stress $𝜎_{xx}$ in the bar. 

2) Consider both the body force $b_f=q$ (N/m) and the concentrated force $f$ (N), derive by hand the formulas for computing the displacement in the $x$-direction, and normal stress $𝜎_{xx}$. Compare the solutions with the corresponding ones obtained using the code given in the textbook. 

3) Given data, $A = 250 mm^2$, $l=0.5m$, Young's modulus of the material $E=2.1e^{10} Pa$, $b_f = 5000 N/m$, $f=1500 N$, compute and plot the distributions of the displacement and stress along the coordinate $x$.

**Question 2:**

Consider a bar of length $L$ consisting of three thin bars of homogeneous isotropic materials, as shown in Fig.\ref{image:BarWith3bars}. The bars are all fixed on a wall at the left-end, and are rigidly connected at the right-ends. The bars on the top and bottom (bar-1) are identical with Young's modulus $E_1$, thermal expansion coefficient of $𝛼_1$, and each with area $A_1$. The bar in the middle (bar-2) is with Young's modulus $E_2$, area $A_2$, and thermal expansion coefficient of $𝛼_2$. Considering only the displacement in the $x$-direction, and complete the following tasks. 

The data are given as $E_1=200 GPa$, $E_2=70 GPa$, $A_1=5mm^2$, $A_2=8mm^2$ and $L=100mm$, $𝛼_1=11.0E^{-6}/ ^∘C$, $𝛼_2=22.0E^{-6}/ ^∘C$, $Δ𝑇=300^∘C$, and $F=50N$. 

<img src="/images/BarWith3bars.png" alt=" " width="400"/><br>

1) Consider only external force $F$ applied at the right-end of the bar, derive the formulas for computing the normal stresses and the internal normal forces in each of the three bars, and the elongation of the bar. 

2) Consider only temperature change $Δ𝑇$ over the entire bars, derive the formulas for computing the normal stresses and the internal normal forces in each of the three bars, and the elongation of the bar. 

3) Using the data given, compute numerical values for normal stresses, internal normal forces in each of the three bars, and the elongation of the bar, when the bar is subjected to both external force and temperature changes. 

4) Compute the principal stresses and the maximum stress and their directions in bar-1. 